In [ ]:
!pip install PyPDF2

## Librerias

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

## Cargar datos desde PDF

In [62]:
import PyPDF2
import os

def cargar_datos_desde_pdf(ruta_pdf):
    """
    Cargar datos desde un archivo PDF y extraer su texto.
    :param ruta_pdf: Ruta del archivo PDF
    :return: Texto extraído del PDF
    """
    texto_completo = ""
    try:
        with open(ruta_pdf, 'rb') as archivo:
            lector_pdf = PyPDF2.PdfReader(archivo)
            for pagina in lector_pdf.pages:
                texto_pagina = pagina.extract_text()
                if texto_pagina:
                    texto_completo += texto_pagina
        return texto_completo
    except FileNotFoundError:
        print("Error: El archivo PDF no se encuentra. Verifica la ruta.")
    except Exception as e:
        print(f"Error al leer el PDF: {e}")
    return None

# Verificar si el archivo existe
ruta_pdf = './Historia.pdf'

if os.path.isfile(ruta_pdf):
    contenido_pdf = cargar_datos_desde_pdf(ruta_pdf)
else:
    print("Error: No se encuentra el archivo PDF.")


## Seleccionamos el Modelo que vamos a utilizar

In [63]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

## Accedemos a la base de datos

In [64]:
vector_store = Chroma(
    collection_name="rag2",
    embedding_function=embeddings,
    persist_directory="./datasets",
)

In [65]:
# Se ve como funciona la base de datos
retriever = vector_store.as_retriever()

## RAG Chain

In [67]:
# Plantilla de conversación
conversation_template = """Por favor, responde la siguiente pregunta basándote solo en el contexto dado:
{context}

Pregunta: {question}
"""

# Crear el prompt a partir de la plantilla
prompt_template = ChatPromptTemplate.from_template(conversation_template)

# LLM local
local_llm = "tinyllama"
local_model = ChatOllama(model=local_llm)

# Cadena de procesamiento
pregunta = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | local_model
    | StrOutputParser()
)

In [71]:
pregunta.invoke("¿Que es un PDF?")

'Sí, un PDF (Portable Document Format) es una estructura de datos abierto que ofrece funcionalidad de carga y descarga de documentos electrónicos en formato de imagen de archivo binario. Se crea con el objetivo de permitar la lectura a través de los dispositivos de visualización electrónico, como computadoras o teléfonos inteligentes, así como su edición y descarga por un equipo personal o en línea, incluida la carga en el servicio de archivos de almacenamiento en el Web Archive (WAR). El término PDF se deriva del símbolo "PDF" (Portable Document Format), que fue desarrollado originalmente por Adobe Systems.'